<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-overview" data-toc-modified-id="Project-overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Project overview</a></span></li><li><span><a href="#Scraping-otodom-link-for-offer-references" data-toc-modified-id="Scraping-otodom-link-for-offer-references-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scraping otodom link for offer references</a></span><ul class="toc-item"><li><span><a href="#Understanding-href-structure" data-toc-modified-id="Understanding-href-structure-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Understanding href structure</a></span></li><li><span><a href="#Extracting-all-offers-from-a-page" data-toc-modified-id="Extracting-all-offers-from-a-page-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Extracting all offers from a page</a></span></li><li><span><a href="#Extracting-all-following-pages" data-toc-modified-id="Extracting-all-following-pages-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Extracting all following pages</a></span></li><li><span><a href="#Create-a-list-of-all-offers" data-toc-modified-id="Create-a-list-of-all-offers-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Create a list of all offers</a></span></li></ul></li><li><span><a href="#Extract-data-from-single-offer" data-toc-modified-id="Extract-data-from-single-offer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extract data from single offer</a></span><ul class="toc-item"><li><span><a href="#Investigate-structure-of-single-offer-page" data-toc-modified-id="Investigate-structure-of-single-offer-page-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Investigate structure of single offer page</a></span></li><li><span><a href="#Drill-down-to-data-stored-in-json-and-strat-exploring-key-dictionaries" data-toc-modified-id="Drill-down-to-data-stored-in-json-and-strat-exploring-key-dictionaries-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Drill down to data stored in json and strat exploring key dictionaries</a></span></li><li><span><a href="#Extract-key-details-into-clear-dictionary" data-toc-modified-id="Extract-key-details-into-clear-dictionary-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Extract key details into clear dictionary</a></span></li></ul></li><li><span><a href="#Combine-offers-into-json-list" data-toc-modified-id="Combine-offers-into-json-list-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Combine offers into json list</a></span><ul class="toc-item"><li><span><a href="#Combine-offers-data-as-single-json-list" data-toc-modified-id="Combine-offers-data-as-single-json-list-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Combine offers data as single json list</a></span></li><li><span><a href="#Save-data-to-local-file-as-JSON-list-and-extract-it-from-local-drive" data-toc-modified-id="Save-data-to-local-file-as-JSON-list-and-extract-it-from-local-drive-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Save data to local file as JSON list and extract it from local drive</a></span></li></ul></li><li><span><a href="#Transforming-JSON-data-to-DataFrame" data-toc-modified-id="Transforming-JSON-data-to-DataFrame-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Transforming JSON data to DataFrame</a></span><ul class="toc-item"><li><span><a href="#Parse-parameters-data" data-toc-modified-id="Parse-parameters-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Parse parameters data</a></span></li><li><span><a href="#Parse-location-data" data-toc-modified-id="Parse-location-data-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Parse location data</a></span></li><li><span><a href="#Parse-characteristics-data" data-toc-modified-id="Parse-characteristics-data-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Parse characteristics data</a></span></li></ul></li><li><span><a href="#Extracting-information-from-lists-and-transorming-data" data-toc-modified-id="Extracting-information-from-lists-and-transorming-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Extracting information from lists and transorming data</a></span><ul class="toc-item"><li><span><a href="#Splitting-features-into-categories" data-toc-modified-id="Splitting-features-into-categories-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Splitting features into categories</a></span></li><li><span><a href="#Categorical-Pipeline" data-toc-modified-id="Categorical-Pipeline-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Categorical Pipeline</a></span><ul class="toc-item"><li><span><a href="#Dealing-with-Null-Values" data-toc-modified-id="Dealing-with-Null-Values-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Dealing with Null Values</a></span></li><li><span><a href="#Unique-categories" data-toc-modified-id="Unique-categories-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Unique categories</a></span></li><li><span><a href="#Multilabel-Features" data-toc-modified-id="Multilabel-Features-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Multilabel Features</a></span></li></ul></li><li><span><a href="#Numeric-pipeline" data-toc-modified-id="Numeric-pipeline-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Numeric pipeline</a></span></li><li><span><a href="#Combining-processed-data-into-final-dataframe" data-toc-modified-id="Combining-processed-data-into-final-dataframe-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Combining processed data into final dataframe</a></span></li></ul></li></ul></div>

In [1]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json
import numpy as np
import timeit 
from datetime import datetime

In [2]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

# Project overview
<a id="project_overview"><a>

# Scraping otodom link for offer references


In [3]:
link="https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26"

In [4]:
html = urlopen(link)
bs = BeautifulSoup(html, 'lxml')

## Understanding href structure
<a id="href_struct"><a>

In [5]:
for item in bs.findAll("a",href=True):
    print(item)
    print("\n\n")

<a class="navbar-brand" href="https://www.otodom.pl" title="Otodom. Najpopularniejszy serwis nieruchomości">www.otodom.pl</a>



<a aria-expanded="false" class="dropdown-toggle" data-toggle="dropdown" data-tracking="classifieds_click" href="https://www.otodom.pl/sprzedaz/" role="button">Ogłoszenia <span class="caret"></span></a>



<a data-observed-action="" data-tracking="classifieds_click" data-tracking-data='{"touch_point_button":"sale_dropdown","touch_point_page": "ads"}' href="https://www.otodom.pl/sprzedaz/">Na sprzedaż</a>



<a data-observed-action="" data-tracking="classifieds_click" data-tracking-data='{"touch_point_button":"rent_dropdown","touch_point_page": "ads"}' href="https://www.otodom.pl/wynajem/">Na wynajem</a>



<a data-observed-action="" data-tracking="developers_click" data-tracking-data='{"touch_point_button":"investments_dropdown", "touch_point_page": "ads"}' href="https://www.otodom.pl/inwestycje/home/">Inwestycje</a>



<a aria-expanded="false" class="dropdown

It seems all offer hrefs contain "www.otodom.pl/oferta/" in their adress. 

The links to move to next page of search results contain "page=" label

In [6]:
bs.findAll("a",href=True)[1].attrs.keys()

dict_keys(['href', 'class', 'data-toggle', 'role', 'aria-expanded', 'data-tracking'])

## Extracting all offers from a page


In [7]:
for item in bs.findAll("a", href=re.compile("www.otodom.pl/oferta/"))[:10]:
    print(item.attrs["href"])
    print("\n")

https://www.otodom.pl/oferta/59m2-wola-jana-kazimierza-bezposrednio-ID45Qh0.html#8e0a67b43c


https://www.otodom.pl/oferta/59m2-wola-jana-kazimierza-bezposrednio-ID45Qh0.html#8e0a67b43c


https://www.otodom.pl/oferta/apartament-w-kamienicy-z-miejscami-postojowymi-ID436OC.html#8e0a67b43c


https://www.otodom.pl/oferta/apartament-w-kamienicy-z-miejscami-postojowymi-ID436OC.html#8e0a67b43c


https://www.otodom.pl/oferta/7pietro-10-200zl-m2-klucze-w-czerwcu-9min-metro-ID45Bp3.html#8e0a67b43c


https://www.otodom.pl/oferta/7pietro-10-200zl-m2-klucze-w-czerwcu-9min-metro-ID45Bp3.html#8e0a67b43c


https://www.otodom.pl/oferta/bezposrednio-nowe-wykonczone-3-pok-0-pcc-garaz-ID45TXb.html#8e0a67b43c


https://www.otodom.pl/oferta/bezposrednio-nowe-wykonczone-3-pok-0-pcc-garaz-ID45TXb.html#8e0a67b43c


https://www.otodom.pl/oferta/lux-apartament-w-kamienicy-bezposrednio-ID45KTI.html#8e0a67b43c


https://www.otodom.pl/oferta/lux-apartament-w-kamienicy-bezposrednio-ID45KTI.html#8e0a67b43c




In [8]:
def get_page_offers(link):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'lxml')
                    
    offer_list=[]

    for item in bs.findAll("a", href=re.compile("www.otodom.pl/oferta/")):
        offer_list.append(item.attrs["href"])
    offer_list=list(set(offer_list))
    return(offer_list)

## Extracting all following pages


In [9]:
for item in bs.findAll("a", href=re.compile("page=")):
    print(item.attrs["href"])
    print("\n")

https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=2


https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=3


https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=500


https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=2




In [10]:
def get_next_page(link):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'lxml')
                    
    next_page=bs.findAll("a", href=re.compile("page="))[0].attrs["href"]

    
    return(next_page)

In [11]:
link

'https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26'

In [12]:
base_next_page=get_next_page(link)
base_next_page=base_next_page[:-1]

In [13]:
page_list=[]
page_list.append(link)

In [14]:
page_list

['https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26']

In [50]:
page_max=500
page_list=[link]
for i in range(2,page_max+1):
#    print(str(i))
    new_page=base_next_page+str(i)
#    print(new_page)
    page_list.append(new_page)

In [51]:
len(page_list)

500

## Create a list of all offers 


In [52]:
all_offers=[]
for i in range(0,len(page_list)-1):

    all_offers=all_offers+get_page_offers(page_list[i])
    if i%50==0:
        print("Scraping offers from link")
        print(page_list[i])
        print("Combined number of offers={}\n".format(len(all_offers)))
    
        

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26
Combined number of offers=30

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=51
Combined number of offers=1530

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=101
Combined number of offers=3030

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=151
Combined number of offers=4530

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=201
Combined number of offers=6030

Scraping offers from link
https://www

In [53]:
len(all_offers)

14969

# Extract data from single offer

## Investigate structure of single offer page 

In [54]:
offer=all_offers[10]

In [55]:
html = urlopen(offer)
bs = BeautifulSoup(html, 'lxml')

In [56]:
bs.title

<title data-react-helmet="true">5 pokoje, mieszkanie na sprzedaż - Warszawa, Mokotów - 60144854 • www.otodom.pl</title>

In [57]:
print(bs.prettify())

<!DOCTYPE html>
<html lang="pl">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="DCr06kqT-gkViKqUArVuJkKTZZ_RDpjNdUtk" name="csrf-token"/>
  <title data-react-helmet="true">
   5 pokoje, mieszkanie na sprzedaż - Warszawa, Mokotów - 60144854 • www.otodom.pl
  </title>
  <meta content="pl" data-react-helmet="true" http-equiv="Content-Language"/>
  <meta content="index, follow" data-react-helmet="true" name="robots"/>
  <meta content="Świetne 5 pokojowe mieszkanie na sprzedaż w miejscowości Warszawa, Mokotów, za cenę 3 500 000 zł. To mieszkanie na sprzedaż  na 3 piętrze ma 162,20 m² powierzchni użytkowej i 162,20 m² powierzchni całkowitej. Biuro nieruchomości POGORZELSKI NIERUCHOMOŚCI jako najważniejsze zalety mieszkania wymienia: garaż/miejsce parkingowe, ogródek, winda. Otodom 60144854" data-react-hel

Afte investigating, where the specific offer price could be found, we can see that it appears most times within sript with "id"= "server-app-state". Let's parse page html to find this script. 

In [58]:
data_script=bs.find("script",{"id":"server-app-state"})

In [59]:
data_script

<script id="server-app-state" type="application/json">{"initialProps":{"meta":{"target":{"Area":"162.2","AreaRange":[],"Build_year":"2001","Building_floors_num":"4","Building_material":["other"],"Building_ownership":["full_ownership"],"Building_type":["apartment"],"City":"warszawa","City_id":"26","Construction_status":["ready_to_use"],"Country":"Polska","Extras_types":["garage","garden","lift","terrace","usable_room"],"Floor_no":["floor_3"],"Heating":["urban"],"Id":"60144854","MarketType":"secondary","Media_types":["cable-television","internet"],"OfferType":"sprzedaz","Photo":"aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=","Price":3500000,"PriceRange":["3000001-10000000"],"Price_per_m":21578,"ProperType":"mieszkanie","Province":"ma

In [60]:
data_script.contents[0]

'{"initialProps":{"meta":{"target":{"Area":"162.2","AreaRange":[],"Build_year":"2001","Building_floors_num":"4","Building_material":["other"],"Building_ownership":["full_ownership"],"Building_type":["apartment"],"City":"warszawa","City_id":"26","Construction_status":["ready_to_use"],"Country":"Polska","Extras_types":["garage","garden","lift","terrace","usable_room"],"Floor_no":["floor_3"],"Heating":["urban"],"Id":"60144854","MarketType":"secondary","Media_types":["cable-television","internet"],"OfferType":"sprzedaz","Photo":"aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=","Price":3500000,"PriceRange":["3000001-10000000"],"Price_per_m":21578,"ProperType":"mieszkanie","Province":"mazowieckie","RegularUser":"n","Rooms_num":["5"],"Secur

## Drill down to data stored in json and strat exploring key dictionaries

In [61]:
data_json=json.loads(data_script.contents[0])

In [62]:
data_json

{'initialProps': {'meta': {'target': {'Area': '162.2',
    'AreaRange': [],
    'Build_year': '2001',
    'Building_floors_num': '4',
    'Building_material': ['other'],
    'Building_ownership': ['full_ownership'],
    'Building_type': ['apartment'],
    'City': 'warszawa',
    'City_id': '26',
    'Construction_status': ['ready_to_use'],
    'Country': 'Polska',
    'Extras_types': ['garage', 'garden', 'lift', 'terrace', 'usable_room'],
    'Floor_no': ['floor_3'],
    'Heating': ['urban'],
    'Id': '60144854',
    'MarketType': 'secondary',
    'Media_types': ['cable-television', 'internet'],
    'OfferType': 'sprzedaz',
    'Photo': 'aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=',
    'Price': 3500000,
    'PriceRange': ['3000

After parsing json dictionary from the chosen sript, we can start to see clear patters. Let's focus on dictionary embedded within ['initialProps'], as it seems to have all the data we need to extract 

In [63]:
#parameters dict
data_json['initialProps']["meta"]["target"]

{'Area': '162.2',
 'AreaRange': [],
 'Build_year': '2001',
 'Building_floors_num': '4',
 'Building_material': ['other'],
 'Building_ownership': ['full_ownership'],
 'Building_type': ['apartment'],
 'City': 'warszawa',
 'City_id': '26',
 'Construction_status': ['ready_to_use'],
 'Country': 'Polska',
 'Extras_types': ['garage', 'garden', 'lift', 'terrace', 'usable_room'],
 'Floor_no': ['floor_3'],
 'Heating': ['urban'],
 'Id': '60144854',
 'MarketType': 'secondary',
 'Media_types': ['cable-television', 'internet'],
 'OfferType': 'sprzedaz',
 'Photo': 'aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=',
 'Price': 3500000,
 'PriceRange': ['3000001-10000000'],
 'Price_per_m': 21578,
 'ProperType': 'mieszkanie',
 'Province': 'mazowieckie',
 

In [64]:
data_json['initialProps']["data"]["advert"]["price"]

{'human_value': '3 500 000 zł',
 'label': 'Cena',
 'unit': 'PLN',
 'value': '3500000'}

In [65]:
#Building characteristics data
data_json['initialProps']["data"]["advert"]['characteristics']

[{'key': 'm',
  'currency': '',
  'label': 'Powierzchnia',
  'value': '162.2',
  'value_translated': '162,20 m²'},
 {'key': 'rooms_num',
  'currency': '',
  'label': 'Liczba pokoi',
  'value': '5',
  'value_translated': '5'},
 {'key': 'market',
  'currency': '',
  'label': 'Rynek',
  'value': 'secondary',
  'value_translated': 'wtórny'},
 {'key': 'building_type',
  'currency': '',
  'label': 'Rodzaj zabudowy',
  'value': 'apartment',
  'value_translated': 'apartamentowiec'},
 {'key': 'floor_no',
  'currency': '',
  'label': 'Piętro',
  'value': 'floor_3',
  'value_translated': '3'},
 {'key': 'building_floors_num',
  'currency': '',
  'label': 'Liczba pięter',
  'value': '4',
  'value_translated': '4'},
 {'key': 'building_material',
  'currency': '',
  'label': 'Materiał budynku',
  'value': 'other',
  'value_translated': 'inne'},
 {'key': 'heating',
  'currency': '',
  'label': 'Ogrzewanie',
  'value': 'urban',
  'value_translated': 'miejskie'},
 {'key': 'build_year',
  'currency': '',

In [66]:
# location data
data_json["initialProps"]["data"]["advert"]['location']

{'address': 'Warszawa, Mokotów, Zbyszka Cybulskiego',
 'coordinates': {'latitude': 52.19973,
  'longitude': 21.04167,
  'radius': 0,
  'zoom_level': 15},
 'geoLevel': [{'id': '7', 'label': 'mazowieckie', 'type': 'region'},
  {'id': '197', 'label': 'Warszawa', 'type': 'sub-region'},
  {'id': '26', 'label': 'Warszawa', 'type': 'city'},
  {'id': '39', 'label': 'Mokotów', 'type': 'district'}]}

In [67]:
offer_dict={}

In [68]:
offer_dict["params"]=data_json['initialProps']["meta"]["target"]
offer_dict["location"]=data_json["initialProps"]["data"]["advert"]['location']
offer_dict["characteristic"]=data_json['initialProps']["data"]["advert"]['characteristics']

## Extract key details into clear dictionary

In [69]:
def get_offer_json(offer):
    html = urlopen(offer)
    bs = BeautifulSoup(html, 'lxml')
    data_script=bs.find("script",{"id":"server-app-state"})
    data_json=json.loads(data_script.contents[0])
    
    offer_dict={}
    offer_dict["params"]=data_json['initialProps']["meta"]["target"]
#    offer_dict["params"]["currency"]=data_json['initialProps']["data"]["advert"]["price"]["unit"]
    offer_dict["location"]=data_json["initialProps"]["data"]["advert"]['location']
    offer_dict["characteristic"]=data_json['initialProps']["data"]["advert"]['characteristics']
    
    return(offer_dict)

In [70]:
get_offer_json(offer)

{'params': {'Area': '162.2',
  'AreaRange': [],
  'Build_year': '2001',
  'Building_floors_num': '4',
  'Building_material': ['other'],
  'Building_ownership': ['full_ownership'],
  'Building_type': ['apartment'],
  'City': 'warszawa',
  'City_id': '26',
  'Construction_status': ['ready_to_use'],
  'Country': 'Polska',
  'Extras_types': ['garage', 'garden', 'lift', 'terrace', 'usable_room'],
  'Floor_no': ['floor_3'],
  'Heating': ['urban'],
  'Id': '60144854',
  'MarketType': 'secondary',
  'Media_types': ['cable-television', 'internet'],
  'OfferType': 'sprzedaz',
  'Photo': 'aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=',
  'Price': 3500000,
  'PriceRange': ['3000001-10000000'],
  'Price_per_m': 21578,
  'ProperType': 'mieszkani

# Combine offers into json list

## Combine offers data as single json list

In [71]:
def create_offers_dict(offers_list):
    combined_offers_list=[]
    for i in range(0, len(offers_list)):
        offer=offers_list[i]
        offer_data={}
        try:
            offer_json=get_offer_json(offer)
            offer_data["Id"]=offer_json["params"]["Id"]
            offer_data["date"]= str(datetime.utcnow().date())
            offer_data["data"]=offer_json
            combined_offers_list.append(offer_data)
        except:
            pass
      
        if i%500==0:
            print("Extracted {}/{} offers at {}".format(i,len(offers_list),str(datetime.now())[:-7]))
    return(combined_offers_list)
        
    

Let's extract all 15k offers - recommend leaving it running for the night as it might take a few hours. 

In [72]:
offers_json=create_offers_dict(all_offers)

Extracted 0/14969 offers at 2020-05-24 20:19:47
Extracted 500/14969 offers at 2020-05-24 20:26:58
Extracted 1000/14969 offers at 2020-05-24 20:34:04
Extracted 1500/14969 offers at 2020-05-24 20:41:33
Extracted 2000/14969 offers at 2020-05-24 20:49:04
Extracted 2500/14969 offers at 2020-05-24 20:56:25
Extracted 3000/14969 offers at 2020-05-24 21:03:56
Extracted 3500/14969 offers at 2020-05-24 21:11:20
Extracted 4000/14969 offers at 2020-05-24 21:18:53
Extracted 4500/14969 offers at 2020-05-24 21:26:15
Extracted 5000/14969 offers at 2020-05-24 21:33:31
Extracted 5500/14969 offers at 2020-05-24 21:41:14
Extracted 6000/14969 offers at 2020-05-24 21:48:30
Extracted 6500/14969 offers at 2020-05-24 21:55:48
Extracted 7000/14969 offers at 2020-05-24 22:02:49
Extracted 7500/14969 offers at 2020-05-24 22:10:06
Extracted 8000/14969 offers at 2020-05-24 22:17:28
Extracted 8500/14969 offers at 2020-05-24 22:24:30
Extracted 9000/14969 offers at 2020-05-24 22:31:23
Extracted 9500/14969 offers at 2020

In [73]:
len(offers_json)

14885

## Save data to local file as JSON list and extract it from local drive

In [74]:
directory="./Warsaw_RE_data/"
if not os.path.exists(directory):
    os.makedirs(directory)
    
todaydate = str(datetime.utcnow().date())
file_name=directory+"Purchase_"+todaydate+".txt"
file_name

'./Warsaw_RE_data/Purchase_2020-05-24.txt'

In [75]:
text_file = open(file_name, "w")
text_file.write(json.dumps(offers_json))
text_file.close()

40788816

In [5]:
file_name="./Warsaw_RE_data/Purchase_2020-05-24.txt"

In [6]:
json_list_txt = open(file_name, "r").read()
offers_json=json.loads(json_list_txt)

# Transforming JSON data to DataFrame

## Parse parameters data

In [7]:
def parse_param_data(json_list):
    param_list=[]
    for i in range(0, len(json_list)):
        parsed_row=json_list[i]["data"]["params"]
        param_list.append(parsed_row)
    
    param_json=json.dumps(param_list)
    df_param=pd.read_json(param_json, orient='Id')
    return(df_param)
  

In [8]:
df_params=parse_param_data(offers_json)
df_params.drop_duplicates(subset="Id",inplace=True)

In [9]:
df_params.head()

,Area,AreaRange,Area_from,Area_to,Build_year,Building_floors_num,Building_material,Building_ownership,Building_type,City,...,Rent,Rooms_num,Security_types,State,Subregion,Title,Windows_type,env,seller_id,user_type
0,80.10,[],NaN,NaN,2021.0,8.0,NaN,NaN,NaN,warszawa,...,NaN,[4],[monitoring],NaN,powiat-warszawa,Tm93b2N6ZcWbbmllIC8gRWxlZ2FuY2tvIC9Lb21mb3J0b3dv,NaN,production,893298.0,professional
1,NaN,NaN,38.0,84.0,NaN,NaN,NaN,NaN,NaN,warszawa,...,NaN,NaN,NaN,[in_building],powiat-warszawa,VmlsbGEgQWxpYW5v,NaN,production,1821192.0,professional
2,46.75,[],NaN,NaN,2019.0,6.0,NaN,[full_ownership],[block],warszawa,...,440.0,[3],"[anti_burglary_door, entryphone]",NaN,powiat-warszawa,TWllc3prYW5pZSAzIHBva29qZSA0Niw3NW0gLSBVcnN1cy...,[plastic],production,NaN,private
3,41.50,[],NaN,NaN,NaN,23.0,NaN,[co_operative_ownership_with_a_land_and_mortga...,[block],warszawa,...,350.0,[1],[entryphone],NaN,powiat-warszawa,TWllc3prYW5pZSB3IENFTlRSVU0gV0FSU1pBV1kgdWxpY2...,NaN,production,NaN,private
4,102.00,[],NaN,NaN,2010.0,3.0,[brick],[full_ownership],[apartment],warszawa,...,700.0,[4],"[entryphone, monitoring]",NaN,powiat-warszawa,RHd1cG96aW9tb3dlLCBrb21mb3J0b3dlLCB6YWRiYW5lIG...,[plastic],production,NaN,private


In [10]:
df_params.shape

(12089, 43)

## Parse location data

In [11]:
json_str=offers_json[0]["data"]["location"]

In [12]:
json_str

{'address': 'Warszawa, Wola, Juliana Konstantego Ordona',
 'coordinates': {'latitude': 52.22124,
  'longitude': 20.95634,
  'radius': 0,
  'zoom_level': 15},
 'geoLevel': [{'id': '7', 'label': 'mazowieckie', 'type': 'region'},
  {'id': '197', 'label': 'Warszawa', 'type': 'sub-region'},
  {'id': '26', 'label': 'Warszawa', 'type': 'city'},
  {'id': '117', 'label': 'Wola', 'type': 'district'}]}

In [13]:
type(json_str["geoLevel"])==list

True

In [14]:
json_str.keys()

dict_keys(['address', 'coordinates', 'geoLevel'])

In [15]:
json_str["coordinates"]['latitude']

52.22124

In [16]:
def extract_location_data(json_entry):
    new_dict={}
    json_str=json_entry["data"]["location"]
    for key in json_str.keys():
        val_level1=json_str[key]
        if type(val_level1)==dict:
            for subkey in val_level1.keys():
                val_level2=val_level1[subkey]
                new_dict[subkey]=val_level2

        elif type(val_level1)==list:
            for i in range(0, len(val_level1)):
                val_level2=val_level1[i]
                new_dict[val_level2["type"]]=val_level2["label"]
        else:

            new_dict[key]=json_str[key]
            
    return(new_dict)


In [17]:
extract_location_data(offers_json[0])

{'address': 'Warszawa, Wola, Juliana Konstantego Ordona',
 'latitude': 52.22124,
 'longitude': 20.95634,
 'radius': 0,
 'zoom_level': 15,
 'region': 'mazowieckie',
 'sub-region': 'Warszawa',
 'city': 'Warszawa',
 'district': 'Wola'}

In [18]:
def parse_location_data(json_list):
    param_list=[]
    for i in range(0, len(json_list)):
        parsed_Id=json_list[i]["Id"]
        parsed_row=extract_location_data(json_list[i])
        parsed_row["Id"]=parsed_Id
        param_list.append(parsed_row)
    
    param_json=json.dumps(param_list)
    df_param=pd.read_json(param_json, orient='Id')
    return(df_param)
  

In [19]:
df_location=parse_location_data(offers_json)
df_location.drop_duplicates(subset="Id",inplace=True)

In [20]:
df_location.head()

,,Id,address,city,district,latitude,longitude,radius,region,sub-region,zoom_level
0,NaN,60534950,"Warszawa, Wola, Juliana Konstantego Ordona",Warszawa,Wola,52.221240,20.956340,0,mazowieckie,Warszawa,15
1,NaN,59816194,"Warszawa, Targówek, Bródno, Bolesławicka 5",Warszawa,Bródno,52.282996,21.031165,0,mazowieckie,Warszawa,15
2,NaN,60534745,"Warszawa, Ursus, Ursus Faktory 2",Warszawa,Ursus,52.197936,20.886419,0,mazowieckie,Warszawa,14
3,NaN,60534948,"Warszawa, Centrum, ul. Świętokrzyska",Warszawa,Centrum,52.235366,21.009301,0,mazowieckie,Warszawa,16
4,NaN,60477253,"Warszawa, Wesoła, Stara Miłosna",Warszawa,Stara Miłosna,52.219170,21.235200,100,mazowieckie,Warszawa,14


In [21]:
df_location.columns

Index(['', 'Id', 'address', 'city', 'district', 'latitude', 'longitude',
       'radius', 'region', 'sub-region', 'zoom_level'],
      dtype='object')

In [22]:
df_location.shape

(12089, 11)

## Parse characteristics data

In [23]:
json_str=offers_json[0]["data"]["characteristic"]

In [24]:
json_str

[{'key': 'm',
  'currency': '',
  'label': 'Powierzchnia',
  'value': '80.1',
  'value_translated': '80,10 m²'},
 {'key': 'rooms_num',
  'currency': '',
  'label': 'Liczba pokoi',
  'value': '4',
  'value_translated': '4'},
 {'key': 'market',
  'currency': '',
  'label': 'Rynek',
  'value': 'primary',
  'value_translated': 'pierwotny'},
 {'key': 'floor_no',
  'currency': '',
  'label': 'Piętro',
  'value': 'floor_1',
  'value_translated': '1'},
 {'key': 'building_floors_num',
  'currency': '',
  'label': 'Liczba pięter',
  'value': '8',
  'value_translated': '8'},
 {'key': 'build_year',
  'currency': '',
  'label': 'Rok budowy',
  'value': '2021',
  'value_translated': '2021'},
 {'key': 'free_from',
  'currency': '',
  'label': 'Dostępne od',
  'value': '2021-12-31',
  'value_translated': '2021-12-31'}]

In [25]:
def parse_characteristic_data(json_list):
    param_list=[]
    for i in range(0, len(json_list)):
        parsed_id=json_list[i]["Id"]
        extracted_dict={}
        parsed_dict=json_list[i]["data"]["characteristic"]
        for j in range(0, len(parsed_dict)):
            parsed_row=parsed_dict[j]
            extracted_dict[parsed_row["key"]]=parsed_row["value"]
        
        extracted_dict["Id"]=parsed_id
            
            
        
        param_list.append(extracted_dict)
    
    param_json=json.dumps(param_list)
    df_param=pd.read_json(param_json, orient='Id')
    return(df_param)
  

In [26]:
df_characteristic=parse_characteristic_data(offers_json)
df_characteristic.drop_duplicates(subset="Id",inplace=True)

In [27]:
df_characteristic.columns

Index(['Id', 'area_from', 'area_to', 'build_year', 'building_floors_num',
       'building_material', 'building_ownership', 'building_type',
       'construction_status', 'floor_no', 'floors_num', 'free_from', 'heating',
       'm', 'market', 'offered_estates_type', 'price_from', 'price_per_m_from',
       'price_to', 'rent', 'rooms_num', 'state', 'windows_type'],
      dtype='object')

In [28]:
df_characteristic.head()

,Id,area_from,area_to,build_year,building_floors_num,building_material,building_ownership,building_type,construction_status,floor_no,...,m,market,offered_estates_type,price_from,price_per_m_from,price_to,rent,rooms_num,state,windows_type
0,60534950,NaN,NaN,2021.0,8.0,NaN,NaN,NaN,NaN,floor_1,...,80.10,primary,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
1,59816194,38.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,flats,357225.0,9300.0,793733.0,NaN,NaN,in_building,NaN
2,60534745,NaN,NaN,2019.0,6.0,NaN,full_ownership,block,ready_to_use,floor_5,...,46.75,secondary,NaN,NaN,NaN,NaN,440.0,3,NaN,plastic
3,60534948,NaN,NaN,NaN,23.0,NaN,co_operative_ownership_with_a_land_and_mortgag...,block,ready_to_use,floor_higher_10,...,41.50,primary,NaN,NaN,NaN,NaN,350.0,1,NaN,NaN
4,60477253,NaN,NaN,2010.0,3.0,brick,full_ownership,apartment,ready_to_use,floor_3,...,102.00,secondary,NaN,NaN,NaN,NaN,700.0,4,NaN,plastic


In [29]:
df_characteristic.shape

(12089, 23)

In [30]:
df_params.to_excel("params.xlsx")
df_location.to_excel("location.xlsx")
df_characteristic.to_excel("characteristic.xlsx")

# Extracting information from lists and transorming data

## Splitting features into categories

In [197]:
df=pd.merge(df_params,df_location,left_on="Id",right_on="Id")
df=pd.merge(df,df_characteristic,left_on="Id",right_on="Id")
df.drop(columns={""},inplace=True)

In [198]:
df=df[df.Price.isna()==False]
df.reset_index(inplace=True)

In [199]:
df.shape

(11939, 75)

In [200]:
#df.to_excel("combined_data.xlsx")

In [201]:
df.columns

Index(['index', 'Area', 'AreaRange', 'Area_from', 'Area_to', 'Build_year',
       'Building_floors_num', 'Building_material', 'Building_ownership',
       'Building_type', 'City', 'City_id', 'Construction_status', 'Country',
       'Equipment_types', 'Extras_types', 'Floor_no', 'Floors_num', 'Heating',
       'Id', 'MarketType', 'Media_types', 'OfferType', 'Offered_estates_type',
       'Photo', 'Price', 'PriceRange', 'Price_from', 'Price_per_m',
       'Price_per_m_from', 'Price_to', 'ProperType', 'Province', 'RegularUser',
       'Rent', 'Rooms_num', 'Security_types', 'State', 'Subregion', 'Title',
       'Windows_type', 'env', 'seller_id', 'user_type', 'address', 'city',
       'district', 'latitude', 'longitude', 'radius', 'region', 'sub-region',
       'zoom_level', 'area_from', 'area_to', 'build_year',
       'building_floors_num', 'building_material', 'building_ownership',
       'building_type', 'construction_status', 'floor_no', 'floors_num',
       'free_from', 'heating', 'm'

In [295]:
num_features=["Area","Price","latitude","longitude","build_year","building_floors_num","rooms_num","floors_num"
]
unique_features=["City","district","market","Building_material","Building_ownership","Building_type","Construction_status","floor_no","Heating",
"Windows_type"]
multi_features=["Equipment_types","Extras_types","Media_types","Security_types"]
cat_features=unique_features+multi_features

## Categorical Pipeline

### Dealing with Null Values

In [296]:
from sklearn.impute import SimpleImputer

In [297]:
class CatImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        """
        Extract unique values from list 

        """ 

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        imp_fill_na = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value="")
        X=pd.DataFrame(imp_fill_na.fit_transform(X),columns=X.columns)
        
            
        return X



In [298]:
imp_fill_na = CatImputer()

In [299]:
df[cat_features]=imp_fill_na.fit_transform(df[cat_features])

### Unique categories

In [300]:
from sklearn.base import BaseEstimator, TransformerMixin

In [301]:
class UniqueAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        """
        Extract unique values from list 

        """ 

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        for feature in X.columns:
            X[feature]=X[feature].apply(lambda x:list_transform(x))
            
        
            
        return X



In [302]:
def list_transform(x):
    if type(x)==list:
        return(x[0])
    else:
        return(x)

In [303]:
unique_adder=UniqueFeaturesTransformer()
df_unique=unique_adder.fit_transform(df[unique_features])
df_unique.head()

,City,district,market,Building_material,Building_ownership,Building_type,Construction_status,floor_no,Heating,Windows_type
0,warszawa,Wola,primary,,,,,floor_1,,
1,warszawa,Ursus,secondary,,full_ownership,block,ready_to_use,floor_5,,plastic
2,warszawa,Centrum,primary,,co_operative_ownership_with_a_land_and_mortgag...,block,ready_to_use,floor_higher_10,urban,
3,warszawa,Stara Miłosna,secondary,brick,full_ownership,apartment,ready_to_use,floor_3,urban,plastic
4,warszawa,Żoliborz,secondary,,,block,to_renovation,floor_2,urban,


### Multilabel Features

In [304]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
mlb = MultiLabelBinarizer()

In [305]:
class MultiAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """ 
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        features=X.columns
        for feature in features:
            feature_one_hot=mlb.fit_transform(X[feature])
            feature_labels=str(feature)+"_"+mlb.classes_
            df_one_hot_feature=pd.DataFrame(feature_one_hot, columns=feature_labels)
            if feature==features[0]:
                df_features=df_one_hot_feature
            else:
                df_features=pd.concat([df_features, df_one_hot_feature], axis=1, sort=False)
        
    
        
       
        
        return(df_features)

In [306]:
multi_adder=MultiAttributesAdder()
df_multi=multi_adder.fit_transform(df[binary_features])


## Numeric pipeline

In [307]:
class NumTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """ 
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        
    
        
       
        
        return(X)

## Combining processed data into final dataframe

In [308]:
df_num=df[num_features]
df_cat=imp_fill_na.fit_transform(df[cat_features])

In [309]:
df_cat_unique=unique_adder.fit_transform(df_cat[unique_features])


In [310]:
unique_cat_pipeline = Pipeline([
    ("imputer", CatImputer()),
    ("attr_adder", UniqueAttributesAdder()),
    ])

multi_cat_pipeline = Pipeline([
    ("imputer", CatImputer()),
    ("attr_adder", MultiAttributesAdder()),
    ])

num_pipeline = Pipeline([
    ("transformer",NumTransformer())
])

In [324]:
df_num=num_pipeline.fit_transform(df[num_features])
df_cat_unique=unique_cat_pipeline.fit_transform(df[unique_features])
df_cat_multi=multi_cat_pipeline.fit_transform(df[multi_features])

In [326]:
df_num.shape

(11939, 8)

In [327]:
df_cat_unique.shape

(11939, 10)

In [328]:
df_cat_multi.shape

(11939, 30)

In [347]:
RE_prices_transformed=pd.concat([df_num, df_cat_unique, df_cat_multi], sort=False, axis=1)

In [348]:
RE_prices_transformed.shape

(11939, 48)

In [349]:
RE_prices_transformed.head()

,Area,Price,latitude,longitude,build_year,building_floors_num,rooms_num,floors_num,City,district,...,Media_types_internet,Media_types_phone,Media_types_sewage,Media_types_water,Security_types_alarm,Security_types_anti_burglary_door,Security_types_closed_area,Security_types_entryphone,Security_types_monitoring,Security_types_roller_shutters
0,80.10,760469.0,52.221240,20.956340,2021.0,8.0,4,NaN,warszawa,Wola,...,0,0,0,0,0,0,0,0,1,0
1,46.75,459000.0,52.197936,20.886419,2019.0,6.0,3,NaN,warszawa,Ursus,...,1,0,0,0,0,1,0,1,0,0
2,41.50,960000.0,52.235366,21.009301,NaN,23.0,1,NaN,warszawa,Centrum,...,0,0,0,0,0,0,0,1,0,0
3,102.00,850000.0,52.219170,21.235200,2010.0,3.0,4,NaN,warszawa,Stara Miłosna,...,1,1,0,0,0,0,0,1,1,0
4,55.00,570000.0,52.265250,20.974110,1960.0,4.0,2,NaN,warszawa,Żoliborz,...,1,1,0,0,0,0,0,1,0,0


In [350]:
RE_prices_transformed.isna().sum()

Area                                     0
Price                                    0
latitude                                 0
longitude                                0
build_year                            2039
building_floors_num                   1095
rooms_num                                0
floors_num                           11939
City                                     0
district                                 0
market                                   0
Building_material                        0
Building_ownership                       0
Building_type                            0
Construction_status                      0
floor_no                                 0
Heating                                  0
Windows_type                             0
Equipment_types_dishwasher               0
Equipment_types_fridge                   0
Equipment_types_furniture                0
Equipment_types_oven                     0
Equipment_types_stove                    0
Equipment_t

In [351]:
RE_prices_transformed.to_excel("Warsaw_RE_data.xlsx")